In [ ]:
include("ABLBreIF/BLBreIF.jl")
import .BLBreIF
include("SCNMF/SCNMF.jl")
import .SCNMF
using Images

In [ ]:
ul = "ORL_Faces/s1/1.pgm"
ig = load(ul)

a, b = size(ig)
R = 5

B = Matrix{Float64}(undef, a*b, 400)
for i = 1 : 40
    for j = 1 : 10
        local url = "ORL_Faces/s"*"$i/"*"$j"*".pgm"
        local img = load(url)
        local q = float64.(channelview(img))
        q = q[:]
        global B[:, (i-1)*10 + j] = q
    end
end
A = SCNMF.normalize!(B)
X, Y = SCNMF.randinit(A, R^2, 0.8)

In [ ]:
ρ₀ = 0.8 # step size parameter
μ₀ = 0.05 # regularization cofficient
μ = 0.005 # another regularization coefficient
rtime = 60 # runtime

In [ ]:
X₀ = copy(X);
Y₀ = copy(Y);
r₀ = SCNMF.solve!(SCNMF.BPG{Float64}(obj=:cons,
                constraint=0.8,
                runtime=rtime,
                ρ=ρ₀), A, X₀, Y₀)

X₁ = copy(X);
Y₁ = copy(Y);
r₁ = SCNMF.solve!(SCNMF.BBPG{Float64}(obj=:cons,
                constraint=0.8,
                runtime=rtime,
                ρ=ρ₀), A, X₁, Y₁)

X₂ = copy(X);
Y₂ = copy(Y);
r₂ = BLBreIF.solve!(BLBreIF.ALBreI{Float64}(runtime=rtime,
                verbose=true,
                ρ=ρ₀,
                μ₁=μ₀,
                μ₂=μ₀), A, X₂, Y₂)

X₃ = copy(X);
Y₃ = copy(Y);
r₃ = BLBreIF.solve!(BLBreIF.ABLBreI{Float64}(runtime=rtime,
                verbose=true,
                ρ=ρ₀,
                μ₁=μ₀,
                μ₂=μ₀), A, X₃, Y₃)
X₄ = copy(X);
Y₄ = copy(Y);
r₄ = BLBreIF.solve!(BLBreIF.ALBreI{Float64}(runtime=rtime,
                verbose=true,
                ρ=ρ₀,
                μ₁=μ,
                μ₂=μ), A, X₄, Y₄)
X₅ = copy(X);
Y₅ = copy(Y);
r₅ = BLBreIF.solve!(BLBreIF.ABLBreI{Float64}(runtime=rtime,
                verbose=true,
                ρ=ρ₀,
                μ₁=μ,
                μ₂=μ), A, X₅, Y₅)

Main.BLBreIF.Result{Float64}([0.037767973423654386 0.009377177636893476 … -0.002007680511841225 0.004360070019382471; -0.008670516397048696 0.04050058987046672 … -0.004691732576167418 0.026095098756930626; … ; 0.018067580959761454 0.02488772811953314 … -0.019014314755823086 0.06260915429645734; -0.0011570077757608824 0.02433881082583809 … 0.017319533568254795 0.037855535030020544], [0.012149003455418117 0.026531415119757314 … 0.029610590459240856 0.028357320572729213; 0.010984302762607527 -0.0279553644604978 … -0.002090613109464698 0.01593766851787229; … ; 0.0 0.05710726393491517 … 0.03299069522099327 -0.005509509610156921; 0.0170605275206301 0.059157154685918124 … 0.03238603392855324 0.002274087779428254], 13287, true, [0.0 0.5348887948561757; 0.0009999275207519531 0.523322263658437; … ; 0.0 0.0; 0.0 0.0])

In [ ]:
stop₀ = r₀.niters
pic₀ = r₀.objvalue
rt₀ = pic₀[1:stop₀, 1]
obj₀ = pic₀[1:stop₀, 2]

stop₁ = r₁.niters
pic₁ = r₁.objvalue
rt₁ = pic₁[1:stop₁, 1]
obj₁ = pic₁[1:stop₁, 2]

stop₂ = r₂.niters
pic₂ = r₂.objvalue
rt₂ = pic₂[1:stop₂, 1]
obj₂ = pic₂[1:stop₂, 2]

stop₃ = r₃.niters
pic₃ = r₃.objvalue
rt₃ = pic₃[1:stop₃, 1]
obj₃ = pic₃[1:stop₃, 2]

stop₄ = r₄.niters
pic₄ = r₄.objvalue
rt₄ = pic₄[1:stop₄, 1]
obj₄ = pic₄[1:stop₄, 2]

stop₅ = r₅.niters
pic₅ = r₅.objvalue
rt₅ = pic₅[1:stop₅, 1]
obj₅ = pic₅[1:stop₅, 2]

In [ ]:
using CairoMakie
using Makie
using LaTeXStrings
using GLMakie
using Colors
using AlgebraOfGraphics
CairoMakie.activate!()
function speed()
        lines(rt₀, sqrt.(2 * obj₀); color="#4063D8", label="BPG", linewidth=3, linestyle=:dashdot,
                figure=(; figure_padding=50, resolution=(1200, 800), font="sans",
                        backgroundcolor=:white, fontsize=32),
                axis=(; xlabel="Time(sec)", ylabel=L"\Vert A-XY\Vert_F",
                        #yscale=log10,
                        xgridstyle=:dash, ygridstyle=:dash))
        lines!(rt₁, sqrt.(2 * obj₁); color="#389826", linewidth=3, linestyle=:dashdot,
                label="BBPG")
        lines!(rt₂, sqrt.(2 * obj₂); color="#FFC633", linewidth=2, linestyle=:solid,
                label="ALBreI(μ=$μ₀)")
        lines!(rt₃, sqrt.(2 * obj₃); color="#CB3C33", linewidth=2, linestyle=:solid,
                label="ABLBreI(μ=$μ₀)")
        lines!(rt₄, sqrt.(2 * obj₄); color="#FFC633", linewidth=2, linestyle=:dot,
                label="ALBreI(μ=$μ)")
        lines!(rt₅, sqrt.(2 * obj₅); color="#CB3C33", linewidth=2, linestyle=:dot,
                label="ABLBreI(μ=$μ)")
        limits!(0, rtime, 0.8, 1.0)
        axislegend("Algorithms"; merge=true)
        current_figure()
end

with_theme(speed, theme=theme_dark())
save("plot/real_data.png", speed())